
The purpose of this notebook is to verify the presence of the expected data files for the ReArm clinical trial. 

The `Rearm` directory contains the experimental files, organized by patient (e.g., `ReArm_C1P02`), then by visit (e.g., `ReArm_C1P02_20210306_V1`), then by type of information (e.g., `Accelerometry`):


    ...
    └── ReArm 
        ├── ReArm_C1P02
        │   ├── ReArm_C1P02_20210306_V1          # first visit of participant 02
        │   │   ├── ReArm_C1P02_20210306_V1_Accelerometry
        │   │   ├── ReArm_C1P02_20210306_V1_Armeo
        │   │   ├── ReArm_C1P02_20210306_V1_Circle
        │   │   ├── ReArm_C1P02_20210306_V1_Reaching
        │   ├   └── ...
        │   ├── ReArm_C1P02_20210419_V2
        │   ├── ...
        ├── ...
        ...

# Usage

1. Run the notebook.
2. Check the output at the end of this notebook. 
3. Optional: manually copy the output and save it to a file for further reference.

# Requirements
- The ReArm dataset
- Python 3.9 or later
- Required packages: `numpy`, `matplotlib`, `pyxdf`, `openmovement`. You can install them by running conda commands below:
```bash
conda create -n Rearm               # Create a new environment
conda activate Rearm                # Activate the environment
conda install numpy matplotlib      # Install numpy and matplotlib
conda install conda-forge::pyxdf    # Install pyxdf from conda-forge (https://anaconda.org/conda-forge/pyxdf)
pip install openmovement            # Install openmovement using pip (https://github.com/digitalinteraction/openmovement-python)
```







# Outputs

The output of this notebook consists in `.log` and `.tsv` files stored in the checked visit directory. The files are:

    ...
    └── ReArm 
        ├── ReArm_C1P02
        │   ├── ReArm_C1P02_20210306_V1         
        │   │   ├── checkFilesInRearmVisit.log   # log file with the summary of checks
        │   │   ├── goodfiles.log                # log file with the expected files that were found
        │   │   ├── kinect_to_mouse_delay.tsv    # log file with delay between kinect and mouse in the .xdf files
        │   │   ├── ReArm_C1P02_20210306_V1_Accelerometry
        │   │   ├── ReArm_C1P02_20210306_V1_Armeo
        │   │   ├── ReArm_C1P02_20210306_V1_Circle
        │   │   ├── ReArm_C1P02_20210306_V1_Reaching
        │   │   └── ...
        │   ├── ...
        ├── ...
        ...



# Code description
## Set the paths

In [ ]:
import os

projectPath = os.path.abspath("..")
dat_path = os.path.join(projectPath, "dat")
res_path = os.path.join(projectPath, "res")
# rearm_path = os.path.join(dat_path, "ReArm.lnk")

In [ ]:
import os
import tkinter as tk
from tkinter import filedialog, messagebox


# Custom exception to stop Jupyter Notebook execution
class StopExecution(Exception):
    pass


# If 'rearm_path' is not in the global scope (outside of any function)
if "rearm_path" not in globals():
    files = os.listdir(dat_path)
    if "ReArm.lnk" in files:
        rearm_path = os.path.join(dat_path, "ReArm.lnk")
        # Verify that the symlink points to a valid folder
        if os.path.exists(rearm_path):
            rearm_path = os.readlink(rearm_path)
    else:
        # Ask for user input if ReArm.lnk is not found
        print("ReArm.lnk not found in dat folder...")
        # Ask the user to select the ReArm folder
        root = tk.Tk()
        root.withdraw()
        rearm_path = filedialog.askdirectory(
            initialdir=dat_path, title="Select the ReArm folder:"
        )

    print(rearm_path)

## Run all the tests

### set the globals

In [ ]:
# set the flag to NOT run the tests
doRunTests = False

# run the outside notebooks (to get the functions)
%run -i "checkFilesInVisit.ipynb"
%run -i "kinect-to-mouse-time-correction.ipynb"
%run -i "checkFileContent.ipynb"
%run -i "renameFilesInVisit.ipynb"


### Get the participants list 

In [ ]:
import pandas as pd

participants_fullname = os.path.join(rearm_path, "participants.tsv")
participants_db = pd.read_csv(participants_fullname, sep="\t")

# # show the participants_db
participants_db.head()

In [ ]:
# ensure that the first column name is 'participant_id'
if participants_db.columns[0] != "participant_id":
    print("The first column name MUST BE 'participant_id' in the participants.tsv file")
    raise StopExecution

# get the columns names into a list
# participants_columns = participants_db.columns.tolist()

# get the participants list and paths
participants_list = participants_db["participant_id"].tolist()
participants_relpath_list = [("ReArm_" + p) for p in participants_list]
participants_abspath_list = [
    os.path.join(rearm_path, p) for p in participants_relpath_list
]

# print the directories one by one
for a, r in zip(participants_abspath_list, participants_relpath_list):
    print(r, " : ", a)

### Run the tests on the Rearm directory


In [ ]:
# create a checkFilesInVisit just to get the checkLog_fname
from numpy import sort


checkLog_fname = CheckFilesInRearmVisit("").checkRes_logFile_name


def alreadyCheckedVisit(visitPath, checkLog_fname):
    """
    Check if the log file already exists, which means that the visit was already checked
    """
    checkLogFullFname = os.path.join(visitPath, checkLog_fname)
    if os.path.isfile(checkLogFullFname):
        print(f"    Already checked visit")
        return True
    return False


print("Selected ReArm folder: " + rearm_path)

# check the files in the Rearm folder
for patient_abspath in participants_abspath_list:
    print("###############################################")
    # split the path to get the patient name
    patient_name = os.path.split(patient_abspath)[-1]
    print(patient_name)
    visits = [
        d
        for d in os.listdir(patient_abspath)
        if os.path.isdir(os.path.join(patient_abspath, d))
    ]
    visit_paths = [
        os.path.join(patient_abspath, visit)
        for visit in visits
        if visit.endswith(("V1", "V2", "V3"))
    ]
    # sort the visits
    visit_paths = sort(visit_paths)
    # visit_paths = [os.path.join(patient_abspath, visit) for visit in visits if visit.endswith(("V1"))]
    for visit_path in visit_paths:
        visit_name = os.path.split(visit_path)[-1]
        print(" ", visit_name)
        if not alreadyCheckedVisit(visit_path, checkLog_fname):
            renameFilesInVisit(visit_path)
            # check the files in the visit
            CheckFilesInRearmVisit(visit_path).run_check()
            check_all_files_content(visit_path)
            # kinectToMouseTimeCorrection(visit_path)

a = 1